In [4]:
from tqdm import tqdm
import sys
import os
import nltk
nltk.download('punkt')  
from nltk.translate.bleu_score import corpus_bleu
sys.path.append(os.path.abspath(os.path.join(os.path.dirname("blue_score.ipynb"), "..")))
from datasets import load_dataset

# from transformers_models.marian.marianMT import 


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Tim\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:

def retrieve_data():
    dataset = load_dataset("wmt14", "de-en")
    
    subset = dataset["train"].select(range(120000))

    # train_data = subset.select(range(100000))
    # val_data = subset.select(range(100000, 110000))
    # test_data = subset.select(range(110000, 120000))


    
    train_data = subset.select(range(5000))
    val_data = subset.select(range(5000, 6000))
    test_data = subset.select(range(6000, 7000))
    return {
        "train": train_data,
        "validation": val_data,
        "test": test_data
    }


In [6]:
def translated(n, model):
    return model.translate_text(n)

In [7]:

data = retrieve_data()


In [8]:
data["validation"]["translation"]

[{'de': 'Dies ist die Intention, die hinter der mündlichen Anfrage steht, und wir müssen dann im Parlament im Rahmen der Berichte in den ordentlichen Verfahren über die Ausschüsse und die Plenarberatungen zu einem ausgewogenen und wirklich zukunftsweisenden Vorschlag kommen.',
  'en': 'This is the intention behind the question and we must then follow the ordinary procedures of reports from committees and plenary debate in order to reach a balanced and really forward-looking proposal.'},
 {'de': 'Frau Präsidentin, meine Damen und Herren! Im Namen der Fraktion der Liberalen und Demokratischen Partei Europas möchte ich mein Erstaunen angesichts der jetzigen Debatte im Zusammenhang mit den internationalen Kapitalströmen und ihrer etwaigen Besteuerung bekunden, was, wenn das auch im Wortlaut nicht erwähnt wird, - wie in der Einführung - ein Versuch der Wiederbelebung der vor einigen Jahren von Herrn Tobin vorgeschlagenen Steuer zu sein scheint, der sich unsere Fraktion in der vergangenen Wa

In [9]:
data["test"]["translation"]

[{'de': 'Von seinem finnischen Vorgänger kann Portugal lernen, wie man es nicht machen soll.',
  'en': "Portugal should not take a leaf out its predecessor' s book."},
 {'de': 'Es ist sehr wichtig, daß das Europäische Parlament und die Kommission bei dieser Regierungskonferenz am selben Strang ziehen.',
  'en': 'It is of the utmost importance that the European Parliament and the Commission join forces for this IGC.'},
 {'de': 'Sie haben in vielen Punkten dieselben Interessen und dieselben Ansichten.',
  'en': 'In many ways, they share the same interests and the same insights.'},
 {'de': 'Ein wesentlicher Bestandteil dieses gemeinsamen Einsatzes muß in jedem Fall das Recht des Parlaments sein, bei den künftigen Vertragsänderungen seine Zustimmung zu geben.',
  'en': "An essential component of this joint effort should, in any case, be this Parliament' s right of assent to the forthcoming changes to the Treaty."},
 {'de': 'Von allen Prioritäten ist dies die wichtigste.',
  'en': 'Of all p

In [10]:
data["train"]["translation"]

[{'de': 'Wiederaufnahme der Sitzungsperiode',
  'en': 'Resumption of the session'},
 {'de': 'Ich erkläre die am Freitag, dem 17. Dezember unterbrochene Sitzungsperiode des Europäischen Parlaments für wiederaufgenommen, wünsche Ihnen nochmals alles Gute zum Jahreswechsel und hoffe, daß Sie schöne Ferien hatten.',
  'en': 'I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period.'},
 {'de': 'Wie Sie feststellen konnten, ist der gefürchtete "Millenium-Bug " nicht eingetreten. Doch sind Bürger einiger unserer Mitgliedstaaten Opfer von schrecklichen Naturkatastrophen geworden.',
  'en': "Although, as you will have seen, the dreaded 'millennium bug' failed to materialise, still the people in a number of countries suffered a series of natural disasters that truly were dreadful."},
 {'de': 'Im Parlament besteht der Wunsch nach einer Aussprache im

In [11]:
from transformers import MarianMTModel, MarianTokenizer
from torch.utils.data import Dataset, DataLoader
import torch

model_name = "Helsinki-NLP/opus-mt-en-de"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

In [12]:
english = [n["en"] for n in data["train"]["translation"]]
german = [n["de"] for n in data["train"]["translation"]]

In [13]:
# translation_pairs_en2de = {"en":[], "de":[]}


# for pair in data["train"]["translation"]:
#     translation_pairs_en2de["en"].append(pair["en"])
#     translation_pairs_en2de["de"].append(pair["de"])

In [14]:
# translation_pairs_en2de

In [15]:
# translation_pairs_en2de_input_output = [
#     {"en": src, "de": tgt}
#     for src, tgt in zip(translation_pairs_en2de["en"], translation_pairs_en2de["de"])
# ]

In [16]:
data

{'train': Dataset({
     features: ['translation'],
     num_rows: 5000
 }),
 'validation': Dataset({
     features: ['translation'],
     num_rows: 1000
 }),
 'test': Dataset({
     features: ['translation'],
     num_rows: 1000
 })}

In [17]:
def preprocess(batch):
    inputs = tokenizer(
        batch["translation"]["en"],
        truncation=True,
        padding="max_length",
        max_length=128
    )
    with tokenizer.as_target_tokenizer():
        targets = tokenizer(
            batch["translation"]["de"],
            truncation=True,
            padding="max_length",
            max_length=128
        )
    inputs["labels"] = targets["input_ids"]
    return inputs


In [18]:
tokenized_data = {
    "train": data["train"].map(preprocess),
    "validation": data["validation"].map(preprocess)
}


In [19]:
from tqdm import tqdm
import evaluate

bleu = evaluate.load("bleu")
test_subset = data["test"].select(range(500))

src_texts = [ex["translation"]["en"] for ex in test_subset]
references = [[ex["translation"]["de"]] for ex in test_subset]

batch_size = 16  
predictions = []

for i in tqdm(range(0, len(src_texts), batch_size), desc="Translating"):
    batch = src_texts[i:i + batch_size]
    inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=128)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=128)
    preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    predictions.extend(preds)


Translating: 100%|██████████| 32/32 [18:50<00:00, 35.34s/it]


In [ ]:
results = bleu.compute(predictions=predictions, references=references)
print(f"BLEU score: {results['bleu']:.4f}")



BLEU score: 0.2337


In [22]:
results

{'bleu': 0.23368769594952757,
 'precisions': [0.5481180811808118,
  0.28620689655172415,
  0.17250996015936254,
  0.1101983237905568],
 'brevity_penalty': 1.0,
 'length_ratio': 1.0403071017274472,
 'translation_length': 13550,
 'reference_length': 13025}

In [23]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./marianmt-finetuned",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=5e-5,
    num_train_epochs=3,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    predict_with_generate=True,
    logging_dir="./logs",
    save_total_limit=2,
    fp16=torch.cuda.is_available(),  
)


c:\Users\Tim\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import Seq2SeqTrainer, DataCollatorForSeq2Seq

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["validation"],
    tokenizer=tokenizer,
    data_collator=DataCollatorForSeq2Seq(tokenizer, model=model),
)

trainer.train()


C:\Users\Tim\AppData\Local\Temp\ipykernel_24728\3911334124.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss


In [ ]:
model.save_pretrained("marianmt-finetuned")
tokenizer.save_pretrained("marianmt-finetuned")
